In [1]:
import requests
from pymongo import MongoClient
import xmltodict
import urllib.request 
import xmljson
from xmljson import badgerfish as bf
from xml.etree.ElementTree import fromstring

In [2]:
import arxiv
from py2neo import Graph

In [3]:
client = MongoClient('mongodb',27017)
db = client['arxiv']
Papers = db["Papers"]

In [47]:
Papers.drop()

In [4]:
## insersion base 
for paper in arxiv.query(query="quantum", max_results=1000):
    Papers.insert_one(paper)

HTTP Error no status in query


In [5]:
uri = "http://neo4j:7474" 
password="esgi_password"
graph = Graph(uri,password=password)
graph.run("MATCH (n) DETACH DELETE n")

In [6]:
cursor_paper = Papers.find()

In [7]:
cursor_paper = Papers.find()

for paper in cursor_paper:
    create_paper = "CREATE (p:PAPER {id: '%s' })" %paper["id"]
    match_authors = ""
    link_authors = ""
    for i, author in enumerate(paper["authors"]):
        match_authors += "MERGE (u%s:AUTHOR {name:\"%s\"}) \n"%(i, author)
        link_authors += "MERGE (u%s)-[:AUTHORED {author_rank: %s}]->(p) \n"%(i, i)
    match_tags = ""
    link_tags = ""
    for i, tag in enumerate(paper["tags"]):
        match_tags += "MERGE (t%s:TAG {name:\"%s\"}) \n"%(i, tag["term"])
        link_tags+= "MERGE (p)-[:TAGGED]->(t%s) \n"%i
    graph.run(create_paper+'\n'+match_authors+'\n'+link_authors+'\n'+match_tags+'\n'+link_tags)
    

In [20]:
def paper_abstract(paper) :
    print("paper url :")
    print(paper["id"])
    print("Title :")
    print(paper["title"])
    print("Summary :")
    print(paper['summary'])
    print("Autors :")
    print(paper["authors"])

In [21]:
def get_paper(author_name, tag):
    c = graph.run("MATCH (a:AUTHOR {name : \"%s\"})-[]-(p:PAPER)-[]-(t:TAG {name: '%s'}) return p"%(author_name,tag ))
    for result in c.data() :
        paper_abstract(Papers.find_one({"id":result['p'].get('id')}))
        print("\n\n")


In [22]:
author_name, tag =  paper['author'], 'quant-ph'


In [48]:
get_paper(author_name, tag)

paper url :


TypeError: 'NoneType' object is not subscriptable

In [45]:
def sorted_colaborators(author_names):
    if isinstance(author_names, str):
        author_names = [author_names]
    dict_author_link = {}
    for name in author_name :
        c = graph.run("MATCH (a:AUTHOR {name : \"%s\"})-[]-(p:PAPER)-[]-(a2:AUTHOR) return a2.name"%(author_name))
        for result in c.data():
            name_bis = result["a2.name"]
            number_author = dict_author_link.get(name_bis, 0)
            number_author+=1
            dict_author_link[name_bis] = number_author
    author_sorted = sorted(list(dict_author_link.items()), key=lambda x : x[1], reverse=True)

    return author_sorted
            

In [46]:
sorted_colaborators(author_name)

[('Ion C. Baianu', 12), ('James F. Glazebrook', 12)]